In [ ]:
# NB-0: 경로/임포트
# pip install pandas openpyxl  (한 번만)
import os, sqlite3, json, re
import pandas as pd
from pathlib import Path

DB_PATH     = "marryroute.db"          # 생성될 SQLite 파일
EXCEL_PATH  = "MarryRoute.DB.xlsx"     # 네 엑셀 경로(파일명/절대경로로 변경 가능)

In [ ]:
# NB-1: 기존 DB 제거(완전 초기화)
if Path(DB_PATH).exists():
    Path(DB_PATH).unlink()
print("DB 초기화 완료:", not Path(DB_PATH).exists())

In [ ]:
# NB-2: 스키마 생성
SCHEMA_SQL = """
PRAGMA foreign_keys = ON;

CREATE TABLE IF NOT EXISTS vendor (
  vendor_id   INTEGER PRIMARY KEY AUTOINCREMENT,
  type        TEXT NOT NULL,          -- hall|studio|dress|makeup
  name        TEXT NOT NULL,          -- 업체명(엑셀 conm)
  region      TEXT,                   -- 지역(엑셀 subway)
  notes       TEXT
);

CREATE TABLE IF NOT EXISTS offering (
  offering_id  INTEGER PRIMARY KEY AUTOINCREMENT,
  vendor_id    INTEGER NOT NULL REFERENCES vendor(vendor_id) ON DELETE CASCADE,
  category     TEXT NOT NULL,         -- hall_package|studio_shoot|dress_rental|makeup_package
  package_name TEXT,                  -- ex) allday|manager|meal_expense ...
  price        INTEGER,               -- KRW(원)
  meta_json    TEXT                   -- 추가정보(JSON)
);

CREATE INDEX IF NOT EXISTS idx_vendor_type   ON vendor(type);
CREATE INDEX IF NOT EXISTS idx_offering_vndr ON offering(vendor_id);
CREATE INDEX IF NOT EXISTS idx_offering_cat  ON offering(category);
"""
with sqlite3.connect(DB_PATH) as conn:
    conn.executescript(SCHEMA_SQL)
print("스키마 생성 완료:", DB_PATH)

In [ ]:
# NB-3: 숫자/스케일/컬럼 유틸
def to_number(v):
    if pd.isna(v): return None
    s = str(v).strip().replace(",", "")
    if s in ("", "-"): return None
    m = re.findall(r"[0-9]+(?:\.[0-9]+)?", s)   # 소수 포함
    return float(m[0]) if m else None

def decide_scale(series):
    # 중앙값<500이면 '만원' 가정 → 원 환산(×10000)
    vals = [to_number(x) for x in series if not pd.isna(x)]
    vals = [v for v in vals if v is not None]
    if not vals: return 1
    vals.sort()
    med = vals[len(vals)//2]
    return 10000 if med < 500 else 1

def pick_col(columns, candidates):
    low = {str(c).strip().lower(): c for c in columns}
    for key in candidates:
        if key in low: return low[key]
    return None

In [ ]:
# NB-4: DB 도우미
def upsert_vendor(conn, vtype, name, region=None, notes=None):
    cur = conn.execute("SELECT vendor_id FROM vendor WHERE type=? AND name=?", (vtype, name))
    row = cur.fetchone()
    if row:
        vid = row[0]
        if region or notes:
            conn.execute("UPDATE vendor SET region=COALESCE(?,region), notes=COALESCE(?,notes) WHERE vendor_id=?",
                         (region, notes, vid))
        return vid
    cur = conn.execute("INSERT INTO vendor(type,name,region,notes) VALUES (?,?,?,?)",
                       (vtype, name, region, notes))
    return cur.lastrowid

def insert_offering(conn, vendor_id, category, package_name, price, meta=None):
    conn.execute(
        "INSERT INTO offering(vendor_id,category,package_name,price,meta_json) VALUES (?,?,?,?,?)",
        (vendor_id, category, package_name, price, json.dumps(meta or {}, ensure_ascii=False))
    )

In [ ]:
# NB-5: wedding_hall / studio
def import_wedding_hall(conn, df):
    name_col   = pick_col(df.columns, ["name","hall_name","conm","unnamed: 0"])
    region_col = pick_col(df.columns, ["region","subway"])
    price_cols = [c for c in ["hall_rental_fee","meal_expense","snapphoto","snapvideo"] if c in df.columns]
    scale = decide_scale(pd.concat([df[c] for c in price_cols], axis=0)) if price_cols else 1

    season_col = pick_col(df.columns, ["season","season(t/f)","seaon(t/f)"])
    peak_col   = pick_col(df.columns, ["peak","peak(t/f)"])
    guar_col   = next((c for c in df.columns if "guarantor" in str(c).lower()), None)

    mapping = {
        "hall_rental_fee": ("hall_package", "hall_rental_fee"),
        "meal_expense":    ("hall_package", "meal_expense"),
        "snapphoto":       ("photo_option", "snapphoto"),
        "snapvideo":       ("video_option", "snapvideo"),
    }

    for _, r in df.iterrows():
        name = str(r[name_col]).strip() if name_col and not pd.isna(r[name_col]) else None
        if not name: continue
        region = str(r[region_col]).strip() if region_col and not pd.isna(r[region_col]) else None
        vid = upsert_vendor(conn, "hall", name, region=region)

        meta = {}
        if season_col: meta["season"] = str(r[season_col]).strip()
        if peak_col:   meta["peak"]   = str(r[peak_col]).strip()
        if guar_col and not pd.isna(r[guar_col]): meta["num_guarantors"] = int(to_number(r[guar_col]))

        for col in price_cols:
            p = to_number(r[col]); price = int(round(p*scale)) if p is not None else None
            cat, pkg = mapping[col]
            insert_offering(conn, vid, cat, pkg, price, meta)

def import_studio(conn, df):
    name_col   = pick_col(df.columns, ["name","studio","conm","unnamed: 0"])
    region_col = pick_col(df.columns, ["region","subway"])
    cands = [("std_price","std"), ("afternoon_price","afternoon"), ("allday_price","allday")]
    price_cols = [c for c,_ in cands if c in df.columns]
    scale = decide_scale(pd.concat([df[c] for c in price_cols], axis=0)) if price_cols else 1

    for _, r in df.iterrows():
        name = str(r[name_col]).strip() if name_col and not pd.isna(r[name_col]) else None
        if not name: continue
        region = str(r[region_col]).strip() if region_col and not pd.isna(r[region_col]) else None
        vid = upsert_vendor(conn, "studio", name, region=region)
        for col, pkg in cands:
            if col in df.columns:
                p = to_number(r[col]); price = int(round(p*scale)) if p is not None else None
                insert_offering(conn, vid, "studio_shoot", pkg, price, {})


In [ ]:
# NB-6: wedding_dress / makeup
def import_wedding_dress(conn, df):
    name_col   = pick_col(df.columns, ["name","dessshop_name","dressshop_name","conm","unnamed: 0"])
    region_col = pick_col(df.columns, ["region","subway"])
    cands = [("wedding","wedding"), ("photo","photo"), ("wedding+photo","wedding+photo"),
             ("fitting_fee","fitting_fee"), ("helper","helper")]
    price_cols = [c for c,_ in cands if c in df.columns]
    scale = decide_scale(pd.concat([df[c] for c in price_cols], axis=0)) if price_cols else 1

    for _, r in df.iterrows():
        name = str(r[name_col]).strip() if name_col and not pd.isna(r[name_col]) else None
        if not name: continue
        region = str(r[region_col]).strip() if region_col and not pd.isna(r[region_col]) else None
        vid = upsert_vendor(conn, "dress", name, region=region)
        for col, pkg in cands:
            if col in df.columns:
                p = to_number(r[col]); price = int(round(p*scale)) if p is not None else None
                insert_offering(conn, vid, "dress_rental", pkg, price, {})

def import_makeup(conn, df):
    name_col   = pick_col(df.columns, ["name","shop","brand","conm","unnamed: 0"])
    region_col = pick_col(df.columns, ["region","subway"])
    role_cols = [(c, c.split("(")[0].strip().lower()) for c in df.columns
                 if any(k in str(c).lower() for k in ["manager","vicedirector","director"])]
    price_cols = [c for c,_ in role_cols]
    scale = decide_scale(pd.concat([df[c] for c in price_cols], axis=0)) if price_cols else 1

    for _, r in df.iterrows():
        name = str(r[name_col]).strip() if name_col and not pd.isna(r[name_col]) else None
        if not name: continue
        region = str(r[region_col]).strip() if region_col and not pd.isna(r[region_col]) else None
        vid = upsert_vendor(conn, "makeup", name, region=region)
        for col, role in role_cols:
            p = to_number(r[col])
            if p is None: continue
            m = re.search(r"\((\d+)\)", str(col))
            meta = {"role": role}
            if m: meta["variant"] = m.group(1)
            price = int(round(p*scale))
            insert_offering(conn, vid, "makeup_package", role, price, meta)

In [ ]:
# NB-7: 엑셀 전체 시트 주입
assert Path(EXCEL_PATH).exists(), f"엑셀을 찾을 수 없습니다: {EXCEL_PATH}"
xlsx = pd.ExcelFile(EXCEL_PATH)
with sqlite3.connect(DB_PATH) as conn:
    conn.execute("PRAGMA foreign_keys = ON;")
    conn.execute("DELETE FROM offering;")
    conn.execute("DELETE FROM vendor;")
    for sheet in xlsx.sheet_names:
        df = xlsx.parse(sheet)
        low = sheet.strip().lower()
        if low in ["wedding_hall","hall"]:
            import_wedding_hall(conn, df);  print(f"[{sheet}] OK")
        elif low in ["studio","studios"]:
            import_studio(conn, df);        print(f"[{sheet}] OK")
        elif low in ["wedding_dress","dress"]:
            import_wedding_dress(conn, df); print(f"[{sheet}] OK")
        elif low in ["makeup","mua"]:
            import_makeup(conn, df);        print(f"[{sheet}] OK")
        else:
            print(f"[{sheet}] 스킵")
    conn.commit()

with sqlite3.connect(DB_PATH) as conn:
    v = conn.execute("SELECT COUNT(*) FROM vendor").fetchone()[0]
    o = conn.execute("SELECT COUNT(*) FROM offering").fetchone()[0]
print(f"✅ Import 완료 | vendors={v}, offerings={o}")


In [ ]:
# NB-8: 조회 함수 (옵션 제외: fitting_fee/helper/snapphoto/snapvideo)
EXCLUDE_FROM_MIN = ("fitting_fee","helper","snapphoto","snapvideo")

def find_vendors(db_path, vtype=None, region=None, price_max=None, keyword=None, limit=10):
    base = f"""
    SELECT v.vendor_id, v.type, v.name, v.region,
    MIN(CASE WHEN o.package_name IS NULL OR LOWER(o.package_name) NOT IN ({",".join(["?"]*len(EXCLUDE_FROM_MIN))})THEN o.price END) AS min_price,
    COUNT(o.offering_id) AS cnt_offers
    FROM vendor v
    LEFT JOIN offering o ON o.vendor_id = v.vendor_id
    WHERE 1=1
    """
    params = [*EXCLUDE_FROM_MIN]
    if vtype:  base += " AND v.type=?"; params.append(vtype)
    if region: base += " AND IFNULL(v.region,'') LIKE ?"; params.append(f"%{region}%")
    if keyword:base += " AND v.name LIKE ?"; params.append(f"%{keyword}%")
    base += " GROUP BY v.vendor_id, v.type, v.name, v.region"

    sql = f"SELECT * FROM ({base}) t"
    if price_max is not None:
        sql += " WHERE t.min_price IS NULL OR t.min_price<=?"
        params.append(price_max)
    sql += " ORDER BY (t.min_price IS NULL), t.min_price, t.name LIMIT ?"
    params.append(limit)

    with sqlite3.connect(db_path) as c:
        c.row_factory = sqlite3.Row
        return [dict(r) for r in c.execute(sql, params).fetchall()]

def get_offerings(db_path, vendor_id):
    sql = """
    SELECT offering_id, category, package_name, price, meta_json
      FROM offering
     WHERE vendor_id=?
     ORDER BY (price IS NULL), price, category, package_name
    """
    with sqlite3.connect(db_path) as c:
        c.row_factory = sqlite3.Row
        rows = c.execute(sql, (vendor_id,)).fetchall()
        out = []
        for r in rows:
            d = dict(r)
            d["meta_json"] = json.loads(d["meta_json"]) if d["meta_json"] else {}
            out.append(d)
        return out

def fmt_price(v):
    if v is None: return "-"
    s = f"{v/10000.0:.1f}".rstrip("0").rstrip(".")
    return f"{s}만원"


In [ ]:
# NB-9: 예시 실행 (필요에 맞게 바꿔봐)
rows = find_vendors(DB_PATH, vtype="dress", region=None, price_max=int(200*10000), limit=5)
for r in rows:
    print(f"#{r['vendor_id']} [{r['type']}] {r['name']} ({r.get('region') or '-'}) | 최저가 {fmt_price(r['min_price'])} | 상품 {r['cnt_offers']}개")

if rows:
    vid = rows[0]['vendor_id']
    print("\n[상세]", rows[0]['name'])
    for o in get_offerings(DB_PATH, vid):
        print("-", o['category'], o['package_name'], ":", fmt_price(o['price']), o['meta_json'])